Word - level CNN sentence classifier 를 만들기 위해서는 Embedding lookup 과 Convolution layer 를 다뤄야 합니다. 

모델을 만들기 전에 먼저 torch.nn.Embedding class 를 다루는 연습을 합니다. (1) torch.nn.Embedding 에서 단어를 lookup 하는 과정과 (2) numpy.ndarray 로 저장되어 있는 word vector 를 torch.nn.Embedding 으로 복사하는 과정을 연습합니다.

우리는 3 차원의 다섯 개 단어의 word vector 를 만듭니다. 

In [1]:
import torch
import torch.nn as nn

import numpy as np
embed_np = np.random.random_sample((5,3))

embed_np

array([[0.99531971, 0.4644181 , 0.69970313],
       [0.88550516, 0.89646713, 0.41505819],
       [0.52155292, 0.28977891, 0.24381698],
       [0.71962643, 0.57947046, 0.39824623],
       [0.63188974, 0.68015251, 0.18565708]])

Embedding vectors 를 이용할 때에는 index 가 없는 단어를 zero vector 로 padding 하는 경우가 있습니다. numpy 에서 zero row 를 만든 뒤, numpy.vstack 을 이용하면 마지막 row 에 zero vector 를 추가할 수 있습니다.

In [2]:
zero_np = np.zeros((1, embed_np.shape[1]), dtype=embed_np.dtype)
embed_np = np.vstack([embed_np, zero_np])
embed_np

array([[0.99531971, 0.4644181 , 0.69970313],
       [0.88550516, 0.89646713, 0.41505819],
       [0.52155292, 0.28977891, 0.24381698],
       [0.71962643, 0.57947046, 0.39824623],
       [0.63188974, 0.68015251, 0.18565708],
       [0.        , 0.        , 0.        ]])

torch.nn.Embedding 을 만들 때에는 num_embeddings 와 embedding_dim 을 지정할 수 있습니다. num_embeddings 는 단어 개수이며, embedding_dim 는 각 단어의 embedding dimension 입니다. 

numpy.ndarray 의 word vector (embed_np) 의 크기를 그대로 복사합니다.

이 때 torch.nn.Embedding 는 random vector 를 만듭니다. 우리가 미리 만들어둔 numpy.ndarray 의 word vector 를 복제합니다. 이를 위해서 torch.from_numpy 를 이용합니다.

```python
embed.weight.data.copy_(torch.from_numpy(embed_np))
```

In [3]:
embed = nn.Embedding(
    num_embeddings = embed_np.shape[0],
    embedding_dim = embed_np.shape[1]
)

print(type(embed.weight.data))

# copy weight from pre-trained weight
embed.weight.data.copy_(torch.from_numpy(embed_np))

<class 'torch.Tensor'>


tensor([[0.9953, 0.4644, 0.6997],
        [0.8855, 0.8965, 0.4151],
        [0.5216, 0.2898, 0.2438],
        [0.7196, 0.5795, 0.3982],
        [0.6319, 0.6802, 0.1857],
        [0.0000, 0.0000, 0.0000]])

Embedding 도 Linear 와 같은 layer 입니다. torch.LongTensor 가 곱해지면, 이에 해당하는 단어들이 lookup 되는 형식입니다. 1 번 단어만을 lookup 하려면 torch.LongTensor([1]) 을 torch.nn.Embedding 에 곱합니다.

In [4]:
# look-up
embed(torch.LongTensor([1]))

tensor([[0.8855, 0.8965, 0.4151]], grad_fn=<EmbeddingBackward>)

여러 개의 단어를 lookup 할 수도 있습니다. [1, 0, 4] 순서로 lookup 을 한 예시입니다. 위의 cell 의 각 단어의 벡터와 그 값이 같습니다.

In [5]:
embed(torch.LongTensor([1,0,4]))

tensor([[0.8855, 0.8965, 0.4151],
        [0.9953, 0.4644, 0.6997],
        [0.6319, 0.6802, 0.1857]], grad_fn=<EmbeddingBackward>)

torch.nn.Embedding 에서 lookup 된 결과 역시 torch.Tensor 입니다. size 는 (3,3) 입니다. size 함수는 numpy.ndarray.shape 과 같은 기능을 합니다.

In [6]:
embed(torch.LongTensor([1,0,4])).size()

torch.Size([3, 3])

3 개의 단어로 이뤄진 2 개의 문장에 대해서도 embedding lookup 이 가능합니다. Batch size 가 2 일 때의 예시입니다.

In [7]:
X = embed(torch.LongTensor(
    [[1,0,4],
     [2,1,5]]))

print(X)
print(X.size())

tensor([[[0.8855, 0.8965, 0.4151],
         [0.9953, 0.4644, 0.6997],
         [0.6319, 0.6802, 0.1857]],

        [[0.5216, 0.2898, 0.2438],
         [0.8855, 0.8965, 0.4151],
         [0.0000, 0.0000, 0.0000]]], grad_fn=<EmbeddingBackward>)
torch.Size([2, 3, 3])


unsqueeze 는 tensor 의 축을 추가하는 것입니다. Convolutional neural network filter 인 Conv 의 forward 함수는 input data 의 size 를 (batch, n channel, image height, image width) 라 가정합니다. 이처럼 input data 의 shape 을 바꿔야 할 때 unsqueeze 가 이용됩니다.

In [9]:
def unsqueeze(X, dim=1):
    return X.unsqueeze(dim=dim)

unsqueeze(X)

tensor([[[[0.8855, 0.8965, 0.4151],
          [0.9953, 0.4644, 0.6997],
          [0.6319, 0.6802, 0.1857]]],


        [[[0.5216, 0.2898, 0.2438],
          [0.8855, 0.8965, 0.4151],
          [0.0000, 0.0000, 0.0000]]]], grad_fn=<UnsqueezeBackward0>)

(2, 3, 3) size 가 (2, 1, 3, 3) 으로 변하였습니다.

In [10]:
unsqueeze(X).size()

torch.Size([2, 1, 3, 3])

squeeze 는 반대로 크기가 1 인 축을 제거하는 것입니다. axis 1 의 크기가 1 이기 때문에 다시 (2, 3, 3) 으로 resize 가 되었습니다.

In [11]:
unsqueeze(X).squeeze(dim=1).size()

torch.Size([2, 3, 3])

만약 크기가 1 보다 큰 axis 에 squeeze 를 실행한다면, tensor 의 size 는 변하지 않습니다.

In [12]:
unsqueeze(X).squeeze(dim=0).size()

torch.Size([2, 1, 3, 3])

이는 view 함수를 이용할 수도 있습니다. reshape 과 같은 기능을 합니다.

In [15]:
def unsqueeze(X, dim=1):
    b, n, m = X.size()
    return X.view(b, 1, n, m)

unsqueeze(X)

tensor([[[[0.8855, 0.8965, 0.4151],
          [0.9953, 0.4644, 0.6997],
          [0.6319, 0.6802, 0.1857]]],


        [[[0.5216, 0.2898, 0.2438],
          [0.8855, 0.8965, 0.4151],
          [0.0000, 0.0000, 0.0000]]]], grad_fn=<ViewBackward>)

In [16]:
unsqueeze(X).size()

torch.Size([2, 1, 3, 3])